In [29]:
import pandas as pd
import string 
import nltk
import re
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

df = pd.read_csv('data/raw data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9185 entries, 0 to 9184
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       9185 non-null   int64  
 1   content  9185 non-null   object 
 2   date     9185 non-null   object 
 3   product  9183 non-null   object 
 4   brand    8988 non-null   object 
 5   rating   9185 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 430.7+ KB


In [30]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words('english')
    output= [i for i in text if i not in stopwords]
    return output

def tokenization(text):
    tokens = re.split('W+',text)
    return tokens

def lemmatizer(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

def clean(text):
    return text[0]

df['clean_content'] = df['content'].apply(lambda x:remove_punctuation(x))
df['clean_content'] = df['clean_content'].apply(lambda x: x.lower())
df['clean_content'] = df['clean_content'].apply(lambda x: tokenization(x))
df['clean_content'] = df['clean_content'].apply(lambda x:remove_stopwords(x))
df['clean_content'] = df['clean_content'].apply(lambda x:lemmatizer(x))
df['clean_content'] = df['clean_content'].apply(lambda x: clean(x))

In [31]:
vocab_size = 20000
max_length = 3000
embedding_dim = 64
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(df['clean_content'])
word_index = tokenizer.word_index
df['sequence'] = tokenizer.texts_to_sequences(df['clean_content'])
df['pad_sequence'] = list(pad_sequences(df['sequence'], maxlen=max_length, padding=padding_type, truncating=trunc_type))

In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3000, 64)          1280000   
                                                                 
 global_average_pooling1d (G  (None, 64)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                1560      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 1,281,585
Trainable params: 1,281,585
Non-trainable params: 0
_________________________________________________________________


2022-06-19 11:56:08.133867: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-19 11:56:08.133987: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-19 11:56:08.134044: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (PU-SV20074): /proc/driver/nvidia/version does not exist
2022-06-19 11:56:08.135328: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])